In [1]:
# import tensorflow as tf
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

In [2]:
print(tf.test.is_built_with_cuda())

True


In [3]:
def prepare_tf_data(Train_size = 32, Test_size = 16, img_size = 224, random_ratio = 0.1):
    (train_ds, test_ds), ds_info = tfds.load(
        'cifar10',
        split=['train', 'test'],
        as_supervised=True,
        with_info=True
    )
    
    def preprocess(image, label):
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.image.resize(image, (img_size, img_size))
        return image, label

    def augment(image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, random_ratio)
        image = tf.image.random_contrast(image, 1 - random_ratio, 1 + random_ratio)
        return image, label
    
    AUTOTUNE = tf.data.AUTOTUNE
    
    train_ds = (train_ds
                .map(preprocess, num_parallel_calls=AUTOTUNE)
                .map(augment, num_parallel_calls=AUTOTUNE)
                .shuffle(10000)
                .batch(Train_size)
                .prefetch(AUTOTUNE))
    
    test_ds = (test_ds
               .map(preprocess, num_parallel_calls=AUTOTUNE)
               .batch(Test_size)
               .prefetch(AUTOTUNE))
    
    return train_ds, test_ds

In [4]:
train_ds, test_ds = prepare_tf_data(Train_size = 128, Test_size = 64,img_size=32)

In [5]:
from models.wideresnet import WideResNet
import tensorflow_addons as tfa

In [ ]:
wide_res = WideResNet(num_classes=10)
# wide_res = tf.keras.models.load_model('./wide_res_cifar10_best')

initial_learning_rate = 1e-2

decay_steps = 1000
lr_schedule_wide_res = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=0.75,  
    staircase=True   
)


# decay_steps = 1000
# lr_schedule_wide_res = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate=initial_learning_rate,
#     decay_steps=decay_steps
# )

wide_res.compile(
    optimizer = tfa.optimizers.AdamW(
    learning_rate=lr_schedule_wide_res,
    weight_decay=0.0001
),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

callbacks_wide_res = [
    tf.keras.callbacks.ModelCheckpoint(
        'wide_res_cifar10_best',  
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.TensorBoard(log_dir='./logs/wide_res_cifar10')
]

# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
# model.fit(..., callbacks=[tensorboard_callback])

history_wide_res = wide_res.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    callbacks=callbacks_wide_res
)

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 1.7490 - accuracy: 0.3660

INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


391/391 [==============================] - 2659s 7s/step - loss: 1.7490 - accuracy: 0.3660 - val_loss: 1.7500 - val_accuracy: 0.3961
Epoch 2/10
391/391 [==============================] - ETA: 0s - loss: 1.2126 - accuracy: 0.5631

INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


...

INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


391/391 [==============================] - 2781s 7s/step - loss: 0.3561 - accuracy: 0.8760 - val_loss: 0.5640 - val_accuracy: 0.8119
Epoch 10/10
391/391 [==============================] - ETA: 0s - loss: 0.3291 - accuracy: 0.8837

INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


391/391 [==============================] - 3032s 8s/step - loss: 0.3291 - accuracy: 0.8837 - val_loss: 0.5425 - val_accuracy: 0.8150


In [ ]:
# Continue Training
import time
wide_res = tf.keras.models.load_model('./wide_res_cifar10_best')

initial_learning_rate = 1e-3

decay_steps = 1000
lr_schedule_wide_res = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=0.75,  
    staircase=True   
)

wide_res.compile(
    optimizer = tfa.optimizers.AdamW(
    learning_rate=lr_schedule_wide_res,
    weight_decay=0.0001
),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

callbacks_wide_res = [
    tf.keras.callbacks.ModelCheckpoint(
        'wide_res_cifar10_best',  
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    log_dir = f"./logs/wide_res_cifar10/{time.strftime('%Y%m%d-%H%M%S')}"
    # tf.keras.callbacks.TensorBoard(log_dir='./logs/wide_res_cifar10')
    tf.keras.callbacks.TensorBoard(log_dir=log_dir)
]


history_wide_res = wide_res.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5,
    callbacks=callbacks_wide_res
)

Epoch 1/5
391/391 [==============================] - ETA: 0s - loss: 0.2149 - accuracy: 0.9261

INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


391/391 [==============================] - 3285s 8s/step - loss: 0.2149 - accuracy: 0.9261 - val_loss: 0.3705 - val_accuracy: 0.8713
Epoch 2/5
391/391 [==============================] - ETA: 0s - loss: 0.1776 - accuracy: 0.9392

INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


...

INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


391/391 [==============================] - 2961s 8s/step - loss: 0.1353 - accuracy: 0.9550 - val_loss: 0.3624 - val_accuracy: 0.8814
Epoch 5/5
391/391 [==============================] - ETA: 0s - loss: 0.1193 - accuracy: 0.9602

INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


INFO:tensorflow:Assets written to: wide_res_cifar10_best\assets


391/391 [==============================] - 3553s 9s/step - loss: 0.1193 - accuracy: 0.9602 - val_loss: 0.3627 - val_accuracy: 0.8819


In [14]:
from models.mobilenet import MobileNet

In [ ]:
# Continue Training
import time
mobile = MobileNet(num_classes=10)

initial_learning_rate = 1e-2

decay_steps = 1000
lr_schedule_mobile = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=0.75,  
    staircase=True   
)

mobile.compile(
    optimizer = tfa.optimizers.AdamW(
    learning_rate=lr_schedule_mobile,
    weight_decay=0.0001
),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

log_dir = f"./logs/mobile_cifar10/{time.strftime('%Y%m%d-%H%M%S')}"
callbacks_mobile = [
    tf.keras.callbacks.ModelCheckpoint(
        './checkpoints/mobile_cifar10_best',  
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    # tf.keras.callbacks.TensorBoard(log_dir='./logs/wide_res_cifar10')
    tf.keras.callbacks.TensorBoard(log_dir=log_dir)
]


history_mobile = mobile.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    callbacks=callbacks_mobile
)

Epoch 1/20
391/391 [==============================] - ETA: 0s - loss: 1.4539 - accuracy: 0.4762

INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


391/391 [==============================] - 75s 188ms/step - loss: 1.4539 - accuracy: 0.4762 - val_loss: 2.3092 - val_accuracy: 0.3835
Epoch 2/20
391/391 [==============================] - ETA: 0s - loss: 1.0188 - accuracy: 0.6373

INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


...

INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


391/391 [==============================] - 204s 519ms/step - loss: 0.2082 - accuracy: 0.9284 - val_loss: 0.6128 - val_accuracy: 0.8117
Epoch 17/20
391/391 [==============================] - 194s 491ms/step - loss: 0.1729 - accuracy: 0.9411 - val_loss: 0.6624 - val_accuracy: 0.8039
Epoch 18/20
391/391 [==============================] - 187s 476ms/step - loss: 0.1601 - accuracy: 0.9465 - val_loss: 0.6630 - val_accuracy: 0.8057
Epoch 19/20
391/391 [==============================] - ETA: 0s - loss: 0.1302 - accuracy: 0.9572

INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


391/391 [==============================] - 211s 537ms/step - loss: 0.1302 - accuracy: 0.9572 - val_loss: 0.6421 - val_accuracy: 0.8173
Epoch 20/20
391/391 [==============================] - 190s 478ms/step - loss: 0.1169 - accuracy: 0.9612 - val_loss: 0.7016 - val_accuracy: 0.8021


In [17]:
from models.squeezenet import SqueezeNet

In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models

def SqueezeNet(input_shape=(32, 32, 3), num_classes=10):
    inputs = tf.keras.Input(input_shape)
    
    x = layers.Conv2D(64, 3, strides=1, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(2, strides=2, padding='same')(x)

    # Fire modules
    def fire_module(x, squeeze_filters, expand_filters):
        squeeze = layers.Conv2D(squeeze_filters, 1, activation='relu')(x)
        expand1x1 = layers.Conv2D(expand_filters, 1, activation='relu')(squeeze)
        expand3x3 = layers.Conv2D(expand_filters, 3, padding='same', activation='relu')(squeeze)
        return layers.Concatenate()([expand1x1, expand3x3])

    x = fire_module(x, 8, 32)
    x = fire_module(x, 8, 32)
    x = layers.MaxPooling2D(2, strides=2, padding='same')(x)  
    x = fire_module(x, 16, 64)
    x = fire_module(x, 16, 64)
   
    x = fire_module(x, 32, 128)
    x = fire_module(x, 32, 128)
    x = layers.GlobalAveragePooling2D()(x) 
    x = layers.Dense(num_classes)(x)
    
    return models.Model(inputs, x)

# model = SqueezeNet(input_shape=(32, 32, 3), num_classes=10)
# model.summary()


In [ ]:
# Continue Training
import time
sq = SqueezeNet(input_shape=(32, 32, 3),num_classes=10)

initial_learning_rate = 1e-3

decay_steps = 1000
lr_schedule_sq = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=0.75,  
    staircase=True  
)

sq.compile(
    optimizer = tfa.optimizers.AdamW(
    learning_rate=lr_schedule_sq,
    weight_decay=0.0001
),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

log_dir = f"./logs/sq_cifar10/{time.strftime('%Y%m%d-%H%M%S')}"
callbacks_sq = [
    tf.keras.callbacks.ModelCheckpoint(
        './checkpoints/sq_cifar10_best',  
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    # tf.keras.callbacks.TensorBoard(log_dir='./logs/wide_res_cifar10')
    tf.keras.callbacks.TensorBoard(log_dir=log_dir)
]


history_sq = sq.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    callbacks=callbacks_sq
)

Epoch 1/20
391/391 [==============================] - ETA: 0s - loss: 1.9797 - accuracy: 0.2341

INFO:tensorflow:Assets written to: ./checkpoints\sq_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\sq_cifar10_best\assets


391/391 [==============================] - 204s 519ms/step - loss: 1.9797 - accuracy: 0.2341 - val_loss: 1.7551 - val_accuracy: 0.3258
Epoch 2/20
391/391 [==============================] - ETA: 0s - loss: 1.6808 - accuracy: 0.3675

INFO:tensorflow:Assets written to: ./checkpoints\sq_cifar10_best\assets


...

INFO:tensorflow:Assets written to: ./checkpoints\sq_cifar10_best\assets


391/391 [==============================] - 135s 346ms/step - loss: 1.1932 - accuracy: 0.5654 - val_loss: 1.1970 - val_accuracy: 0.5602
Epoch 18/20
391/391 [==============================] - ETA: 0s - loss: 1.1925 - accuracy: 0.5647

INFO:tensorflow:Assets written to: ./checkpoints\sq_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\sq_cifar10_best\assets


391/391 [==============================] - 134s 343ms/step - loss: 1.1925 - accuracy: 0.5647 - val_loss: 1.1906 - val_accuracy: 0.5667
Epoch 19/20
391/391 [==============================] - 134s 343ms/step - loss: 1.1980 - accuracy: 0.5642 - val_loss: 1.1857 - val_accuracy: 0.5638
Epoch 20/20
391/391 [==============================] - 135s 344ms/step - loss: 1.2038 - accuracy: 0.5616 - val_loss: 1.2019 - val_accuracy: 0.5607


In [42]:
def train_AdamW_ExpDecay(config = None, checkpoint = None, activate_fun = True, epochs=20, patience = 5, initial_lr = 1e-3, decay_steps = 1000, 
        decay_rate = 0.9, weight_decay = 0.0001, train_ds = train_ds, val_ds = test_ds, model_name = 'model', data_name = 'cifar10'):
    import time

    if config:
        model = config
    elif checkpoint:
        model = tf.keras.models.load_model(checkpoint)
    else:
        raise ValueError("Either 'config' or 'checkpoint' must be provided.")


    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_lr,
        decay_steps=decay_steps,
        decay_rate=decay_rate,  
        staircase=True   
    )

    model.compile(
        optimizer = tfa.optimizers.AdamW(
        learning_rate=lr_schedule,
        weight_decay=weight_decay
    ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=activate_fun),
        metrics=['accuracy']
    )

    log_dir = "./logs/" + model_name + "_" + data_name + f"/{time.strftime('%Y%m%d-%H%M%S')}"
    checkpoint_dir = "./checkpoints/" + model_name + "_" + data_name + "_best"
    # log_dir = f"./logs/sq_cifar10/{time.strftime('%Y%m%d-%H%M%S')}"

    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            #'./checkpoints/sq_cifar10_best',  
            checkpoint_dir,
            save_best_only=True,
            monitor='val_accuracy'
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=patience,
            restore_best_weights=True
        ),
        # tf.keras.callbacks.TensorBoard(log_dir='./logs/wide_res_cifar10')
        tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks
    )

    return history

In [36]:
train_AdamW_ExpDecay(checkpoint = './checkpoints/mobile_cifar10_best', activate_fun = True, epochs=10, patience = 5, initial_lr = 5e-4, decay_steps = 500, 
        decay_rate = 0.8, weight_decay = 0.0001, train_ds = train_ds, val_ds = test_ds, model_name = 'mobile', data_name = 'cifar10')

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 0.0897 - accuracy: 0.9752

INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


391/391 [==============================] - 99s 249ms/step - loss: 0.0897 - accuracy: 0.9752 - val_loss: 0.6257 - val_accuracy: 0.8138
Epoch 2/10
391/391 [==============================] - ETA: 0s - loss: 0.0711 - accuracy: 0.9827

INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


391/391 [==============================] - 101s 259ms/step - loss: 0.0711 - accuracy: 0.9827 - val_loss: 0.6224 - val_accuracy: 0.8194
Epoch 3/10
391/391 [==============================] - 95s 243ms/step - loss: 0.0650 - accuracy: 0.9843 - val_loss: 0.6337 - val_accuracy: 0.8184
Epoch 4/10
391/391 [==============================] - 96s 246ms/step - loss: 0.0579 - accuracy: 0.9880 - val_loss: 0.6259 - val_accuracy: 0.8190
Epoch 5/10
391/391 [==============================] - 96s 245ms/step - loss: 0.0550 - accuracy: 0.9890 - val_loss: 0.6287 - val_accuracy: 0.8181
Epoch 6/10
391/391 [==============================] - ETA: 0s - loss: 0.0515 - accuracy: 0.9909

INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\mobile_cifar10_best\assets


391/391 [==============================] - 98s 249ms/step - loss: 0.0515 - accuracy: 0.9909 - val_loss: 0.6250 - val_accuracy: 0.8199
Epoch 7/10
391/391 [==============================] - 96s 245ms/step - loss: 0.0503 - accuracy: 0.9921 - val_loss: 0.6274 - val_accuracy: 0.8186
Epoch 8/10
391/391 [==============================] - 97s 247ms/step - loss: 0.0494 - accuracy: 0.9932 - val_loss: 0.6160 - val_accuracy: 0.8188
Epoch 9/10
391/391 [==============================] - 96s 245ms/step - loss: 0.0502 - accuracy: 0.9935 - val_loss: 0.6115 - val_accuracy: 0.8197
Epoch 10/10
391/391 [==============================] - 95s 243ms/step - loss: 0.0509 - accuracy: 0.9941 - val_loss: 0.6042 - val_accuracy: 0.8196


In [37]:
from models.efficientnet import EfficientNetB0

In [53]:
train_AdamW_ExpDecay(checkpoint= './checkpoints/eff_cifar10_best', activate_fun = False, epochs=10, patience = 5, initial_lr = 1e-4, decay_steps = 1000, 
        decay_rate = 0.75, weight_decay = 0.0001, train_ds = train_ds, val_ds = test_ds, model_name = 'eff', data_name = 'cifar10')

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 1.1693 - accuracy: 0.5770

INFO:tensorflow:Assets written to: ./checkpoints\eff_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\eff_cifar10_best\assets


391/391 [==============================] - 572s 1s/step - loss: 1.1693 - accuracy: 0.5770 - val_loss: 1.3288 - val_accuracy: 0.5356
Epoch 2/10
391/391 [==============================] - ETA: 0s - loss: 1.1646 - accuracy: 0.5783

INFO:tensorflow:Assets written to: ./checkpoints\eff_cifar10_best\assets


INFO:tensorflow:Assets written to: ./checkpoints\eff_cifar10_best\assets


391/391 [==============================] - 926s 2s/step - loss: 1.1646 - accuracy: 0.5783 - val_loss: 1.3116 - val_accuracy: 0.5416
Epoch 3/10
391/391 [==============================] - 585s 1s/step - loss: 1.1633 - accuracy: 0.5800 - val_loss: 1.3331 - val_accuracy: 0.5325
Epoch 4/10
391/391 [==============================] - 610s 2s/step - loss: 1.1607 - accuracy: 0.5814 - val_loss: 1.3305 - val_accuracy: 0.5341
Epoch 5/10
391/391 [==============================] - 608s 2s/step - loss: 1.1591 - accuracy: 0.5831 - val_loss: 1.3412 - val_accuracy: 0.5339
Epoch 6/10
391/391 [==============================] - 601s 2s/step - loss: 1.1604 - accuracy: 0.5803 - val_loss: 1.3637 - val_accuracy: 0.5245
Epoch 7/10
391/391 [==============================] - 593s 2s/step - loss: 1.1634 - accuracy: 0.5794 - val_loss: 1.3527 - val_accuracy: 0.5246
